<a href="https://colab.research.google.com/github/rishalfanda/vton-cnn-transformer-warping/blob/main/Thesis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Drive and Load Dataset

In [10]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()


In [2]:
# Colab bootstrap: mount Drive and write auto.yml
from google.colab import drive
drive.mount('/content/drive')
print('Drive mounted. Create /content/configs/auto.yml by running the bootstrap in your local environment if needed.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted. Create /content/configs/auto.yml by running the bootstrap in your local environment if needed.


In [2]:
%%capture
!unzip /content/drive/MyDrive/dataset/cloth.zip -d ./dataset-cloth
!unzip /content/drive/MyDrive/dataset/cloth_train.zip -d ./dataset-cloth_train
!unzip /content/drive/MyDrive/dataset/image.zip -d ./dataset-image
!unzip /content/drive/MyDrive/dataset/image_train.zip -d ./dataset-image_train
!unzip /content/drive/MyDrive/dataset/cloth-mask.zip -d ./dataset-cloth-mask

!unzip /content/drive/MyDrive/dataset/image-parse-v3.zip -d ./dataset-image-parse-v3
!unzip /content/drive/MyDrive/dataset/openpose_img.zip -d ./dataset-openpose_img
!unzip /content/drive/MyDrive/dataset/openpose_img-test.zip -d ./dataset-openpose_img-test
!unzip /content/drive/MyDrive/dataset/image-parse-v3-test.zip -d ./dataset-image-parse-v3-test
!unzip /content/drive/MyDrive/dataset/cloth-mask-test.zip -d ./dataset-cloth-mask-test
!unzip /content/drive/MyDrive/dataset/openpose_json.zip -d ./dataset-openpose_json
!unzip /content/drive/MyDrive/dataset/openpose_json_train.zip -d ./dataset-openpose_json_train

In [ ]:
!rm -rf /content/vton_dataset/openpose_json

In [3]:
!mkdir vton_dataset
!mkdir vton_dataset/cloth
!mv /content/dataset-cloth_train/cloth vton_dataset/cloth/cloth_train
!rm -rf /content/dataset-cloth_train
!mv /content/dataset-cloth/cloth vton_dataset/cloth/cloth_test
!rm -rf /content/dataset-cloth

!mkdir vton_dataset/image
!mv /content/dataset-image_train/image vton_dataset/image/image_train
!rm -rf /content/dataset-image_train
!mv /content/dataset-image/image vton_dataset/image/image_test
!rm -rf /content/dataset-image

!mkdir vton_dataset/cloth_mask
!mv /content/dataset-cloth-mask/cloth-mask vton_dataset/cloth_mask/cloth_mask_train
!rm -rf /content/dataset-cloth-mask
!mv /content/dataset-cloth-mask-test/cloth-mask vton_dataset/cloth_mask/cloth_mask_test
!rm -rf /content/dataset-cloth-mask-test

!mkdir vton_dataset/image_parse
!mv /content/dataset-image-parse-v3/image-parse-v3 vton_dataset/image_parse/image_parse_train
!rm -rf /content/dataset-image-parse-v3
!mv /content/dataset-image-parse-v3-test/image-parse-v3 vton_dataset/image_parse/image_parse_test
!rm -rf /content/dataset-image-parse-v3-test

!mkdir vton_dataset/openpose_img
!mv /content/dataset-openpose_img/openpose_img vton_dataset/openpose_img/openpose_img_train
!rm -rf /content/dataset-openpose_img
!mv /content/dataset-openpose_img-test/openpose_img vton_dataset/openpose_img/openpose_img_test
!rm -rf /content/dataset-openpose_img-test

!mkdir vton_dataset/openpose_json
!mv /content/dataset-openpose_json_train/openpose_json vton_dataset/openpose_json/openpose_json_train
!rm -rf /content/dataset-openpose_json_train/
!mv /content/dataset-openpose_json/openpose_json vton_dataset/openpose_json/openpose_json_test
!rm -rf /content/dataset-openpose_json/

In [3]:
# @title
import os

base = "vton_dataset"
folders = [
    "cloth/cloth_train", "cloth/cloth_test",
    "image/image_train", "image/image_test",
    "cloth_mask/cloth_mask_train", "cloth_mask/cloth_mask_test",
    "image_parse/image_parse_train", "image_parse/image_parse_test",
    "openpose_img/openpose_img_train", "openpose_img/openpose_img_test",
    "openpose_json/openpose_json_train", "openpose_json/openpose_json_test"
]

for f in folders:
    print(f, len(os.listdir(os.path.join(base, f))))

cloth/cloth_train 11647
cloth/cloth_test 2032
image/image_train 11647
image/image_test 2032
cloth_mask/cloth_mask_train 11647
cloth_mask/cloth_mask_test 2032
image_parse/image_parse_train 11647
image_parse/image_parse_test 2032
openpose_img/openpose_img_train 11647
openpose_img/openpose_img_test 2032
openpose_json/openpose_json_train 11647
openpose_json/openpose_json_test 2032


In [ ]:
# !git clone https://github.com/rishalfanda/vton-cnn-transformer-warping.git

Cloning into 'vton-cnn-transformer-warping'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 21.01 KiB | 7.00 MiB/s, done.


#Config and Preprocessing

In [4]:
import sys
sys.path.append("/content/vton_project")

from dataloader.vton_data_loader import create_dataloaders
from configs.vton_config_default import config

# Update the data_root in the config to point to the correct location
config["dataset"]["data_root"] = "/content/vton_dataset"

train_loader, val_loader = create_dataloaders(config, batch_size_override=2)
batch = next(iter(train_loader))
print("Batch keys:", batch.keys())
print("person_img:", batch['person_img'].shape)
if 'parse' in batch: print("parse:", batch['parse'].shape)
if 'pose' in batch: print("pose:", batch['pose'].shape)

INFO:VITON-Dataloader:VITONDataset mode=train loaded 11647 samples from /content/vton_dataset
INFO:VITON-Dataloader:VITONDataset mode=test loaded 2032 samples from /content/vton_dataset


Batch keys: dict_keys(['person_img', 'cloth_img', 'person_id', 'cloth_id', 'parse', 'pose', 'cloth_mask'])
person_img: torch.Size([2, 3, 512, 384])
parse: torch.Size([2, 512, 384])
pose: torch.Size([2, 18, 512, 384])


#Build Model (U-Net & TransUnet)

In [41]:
!head -n 50 /content/vton_project/models/vton_unet.py


import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_ch, out_ch)
        )
    def forward(self, x):
        return self.net(x)

class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, 2, stride=2)
        self.conv = DoubleConv(in_ch, out_ch)
    def forward(self, x1, x2):
       

In [5]:
import sys, torch
sys.path.append("/content/vton_project")

from dataloader.vton_data_loader import create_dataloaders
from configs.vton_config_default import config
from utils.input_builder import build_input
import importlib

import models.vton_unet
importlib.reload(models.vton_unet)

from models.vton_unet import UNet


# Load data
train_loader, test_loader = create_dataloaders(config)

batch = next(iter(train_loader))
x = build_input(batch, use_cloth=False, use_cloth_mask=False)  # (B,22,H,W)
print("Input shape:", x.shape)

# Model
model = UNet(in_channels=x.shape[1], num_classes=20)
out = model(x)
print("Output shape:", out.shape)


INFO:VITON-Dataloader:VITONDataset mode=train loaded 11647 samples from /content/vton_dataset
INFO:VITON-Dataloader:VITONDataset mode=test loaded 2032 samples from /content/vton_dataset


Input shape: torch.Size([2, 22, 512, 384])
Output shape: torch.Size([2, 20, 512, 384])


In [8]:
import sys, torch
sys.path.append("/content/vton_project")

import models.vton_transunet
importlib.reload(models.vton_transunet)

from models.vton_transunet import TransUNetLight


model = TransUNetLight(in_channels=22, num_classes=20)
x = torch.randn(2, 22, 256, 192)
y = model(x)
print("Output shape:", y.shape)



Output shape: torch.Size([2, 20, 256, 192])


#Training

In [17]:
!python /content/vton_project/training/train_vton.py


Device: cuda
/content/vton_project/training/train_vton.py:87: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/content/vton_project/training/train_vton.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:409: operator(): block: [70,0,0], thread: [0,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:409: operator(): block: [70,0,0], thread: [1,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:409: operator(): block: [70,0,0], thread: [2,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/pytorch/aten/src/ATen/nat

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (ipython-input-266447726.py, line 4)

In [19]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [20]:
!git config --global user.name "rishalfanda"
!git config --global user.email "rizalalfanda@gmail.com"

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
